In [3]:
!pip install selenium
from selenium import webdriver

In [24]:
from tqdm import tqdm_notebook as tqdmn

In [5]:
import pandas as pd

In [8]:
!pip install folium
import folium

In [10]:
pd.set_option('display.max_columns', 60)

acnc = pd.read_excel('./datadotgov_main.xlsx', keep_default_na=False)
acnc.head()

,ABN,Charity_Legal_Name,Other_Organisation_Names,Operating_Countries,Address_Type,Address_Line_1,Address_Line_2,Address_Line_3,Town_City,State,Postcode,Country,Charity_Website,Registration_Date,Date_Organisation_Established,Charity_Size,Number_of_Responsible_Persons,Financial_Year_End,Operates_in_ACT,Operates_in_NSW,Operates_in_NT,Operates_in_QLD,Operates_in_SA,Operates_in_TAS,Operates_in_VIC,Operates_in_WA,PBI,HPC,Preventing_or_relieving_suffering_of_animals,Advancing_Culture,Advancing_Education,Advancing_Health,Promote_or_oppose_a_change_to_law__government_poll_or_prac,Advancing_natual_environment,Promoting_or_protecting_human_rights,Purposes_beneficial_to_ther_general_public_and_other_analogous,Promoting_reconciliation__mutual_respect_and_tolerance,Advancing_Religion,Advancing_social_or_public_welfare,Advancing_security_or_safety_of_Australia_or_Australian_public,Another_purpose_beneficial_to_the_community,Aboriginal_or_TSI,Aged_Persons,Children,Communities_Overseas,Ethnic_Groups,Gay_Lesbian_Bisexual,General_Community_in_Australia,Men,Migrants_Refugees_or_Asylum_Seekers,Pre_Post_Release_Offenders,People_with_Chronic_Illness,People_with_Disabilities,People_at_risk_of_homelessness,Unemployed_Person,Veterans_or_their_families,Victims_of_crime,Victims_of_Disasters,Women,Youth
0,11000047950,Sydney Missionary & Bible College,,,Business,43 Badminton Rd,,,Croydon,New South Wales,2132,Australia,,03/12/2012,,Large,9,31-Dec,,Y,,,,,,,,,,,Y,,,,,,,Y,,,,,,,Y,Y,,Y,,,,,,,,,,,,
1,11000073870,Integricare,,,Business,,,,,,,Australia,http://www.integricare.org.au,03/12/2012,31/12/1881,Large,8,30-Jun,,Y,,,,,,,,,,,,,,,,Y,,,Y,,Y,,,,,,,,,,,,,,,,,,,
2,11000761571,Australian Missionary Tidings,,"BRA, COD, COL, ESP, FJI, IND, ITA, KHM, MEX, N...",Business,Po Box 565,,,Mount Gravatt,Queensland,4122,Australia,,03/12/2012,31/12/1909,Large,12,31-Dec,,,,Y,,,,,,,,,,,,,,,,Y,,,,,,,Y,Y,,,,,,,,,,,,Y,,
3,11001233790,"The Greek Orthodox Church & Community Of The ""...",,,Business,11 Steel Street,,,HAMILTON,NSW,2303,Australia,,03/12/2012,,Medium,5,30-Jun,,Y,,,,,,,,,,,,,,,,,,Y,,,,Y,,,Y,Y,Y,Y,Y,Y,,,Y,,,,,Y,Y,Y
4,11001241005,Wangarang Industries Limited,,,Business,PO Box 8133,,,Orange,NSW,2800,Australia,,03/12/2012,01/01/1975,Large,8,30-Jun,,Y,,,,,,,Y,,,,,,,,,,,,,,,,,,,,,,,,,,Y,,,,,,,


In [11]:
mel = acnc[acnc.Town_City.str.contains('melbourne', case=False)][['ABN', 'Charity_Legal_Name', 'Address_Line_1', 'Address_Line_2', 'Address_Line_3', 'Town_City', 'State', 'Postcode', 'Country', 'Date_Organisation_Established', 'Charity_Size']].copy()

In [12]:
mel.Town_City.value_counts()

Melbourne                       1785
MELBOURNE                        473
East Melbourne                   115
South Melbourne                  105
North Melbourne                   89
Port Melbourne                    47
West Melbourne                    33
SOUTH MELBOURNE                   20
EAST MELBOURNE                    18
NORTH MELBOURNE                    8
PORT MELBOURNE                     5
WEST MELBOURNE                     5
Royal Melbourne Hospital           4
Melbourne University               3
University of Melbourne            2
Melbourne Airport                  2
University Of Melbourne            2
The Royal Melbourne Hospital       1
Melbourne Officer                  1
UNIVERSITY OF MELBOURNE            1
THE UNIVERSITY OF MELBOURNE        1
The University of Melbourne        1
Melbourne Aiport                   1
melbourne                          1
Hawthorne East, Melbourne          1
Melbourne Law Courts               1
Melbourne North                    1
W

In [13]:
mel.head()

,ABN,Charity_Legal_Name,Address_Line_1,Address_Line_2,Address_Line_3,Town_City,State,Postcode,Country,Date_Organisation_Established,Charity_Size
15,11004285956,Australian Nurses Memorial Centre,G Se 11 431 St Kilda Rd,,,Melbourne,Victoria,3004,Australia,31/12/1949,Small
123,11128866419,Australian Community Philanthropy Ltd,L 15 1 Collins St,,,Melbourne,Victoria,3000,Australia,01/01/2007,Small
125,11129013305,Nafsika Stamoulis Hellenic Museum Limited,280 William St,,,Melbourne,Victoria,3000,Australia,01/01/2007,Large
126,11129790467,Doxa Community Club Incorporated,"Ground Floor, Suite 2, 355 Spencer Street",,,West Melbourne,VIC,3003,Australia,01/01/1992,Large
135,11137037677,Financial Basics Community Foundation Limited,L 7 525 Flinders St,,,Melbourne,Victoria,3000,Australia,01/01/2009,Small


In [15]:
mel['Full_Address'] = mel['Address_Line_1'].str.cat(mel[['Address_Line_2', 'Address_Line_3', 'Town_City']], sep=' ')

In [16]:
mel.Full_Address.iloc[0]

'G Se 11 431 St Kilda Rd   Melbourne'

In [17]:
mel[mel.Full_Address.str.contains('po box', case=False)].Full_Address.iloc[0]

'GPO Box 2307   Melbourne'

In [18]:
mel = mel[~mel.Full_Address.str.contains('po box', case=False)].copy()

In [19]:
mel.Full_Address = mel.Full_Address.str.replace('/', ' ')

In [21]:
mel['Url'] = ['https://www.google.com/maps/search/' + i for i in mel['Full_Address']]

In [25]:
Url_With_Coordinates = []

option = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': {'images':2, 'javascript':2}}
option.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome("D:\\chromedriver.exe", options=option)

for url in tqdmn(mel.Url, leave=False):
    driver.get(url)
    Url_With_Coordinates.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))

In [26]:
driver.close()

In [27]:
import csv

with open('Url_With_Coordinates.csv', 'w') as file:
    wr = csv.writer(file)
    wr.writerow(Url_With_Coordinates)

In [29]:
import csv
with open('Url_With_Coordinates.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for i in reader:
        Url_With_Coordinates = i
        break

In [30]:
Url_With_Coordinates

['https://maps.google.com/maps/api/staticmap?center=-37.8386737%2C144.97706&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=uFBzLNXh4_xAkoqxe9tXB97_zFk',
 'https://maps.google.com/maps/api/staticmap?center=-37.8139436%2C144.9734554&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=ZDtMkZz6Vw066nVDdUtFWzsWuuE',
 'https://maps.google.com/maps/api/staticmap?center=-37.8125657%2C144.9570751&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=KWS2gZbSdewzujGQh9qmodbP9Sc',
 'https://maps.google.com/maps/api/staticmap?center=-37.8119391%2C144.950558&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=Ma5nOuERyqGe7J9Ji6P2WXUNnpA',
 'https://maps.google.com/maps/api/staticmap?center=-37.820758%2C144.9571827&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=FAylt0yX0iJEynnvARdN2WirFR8',
 'https://maps.google.com/maps/api/staticmap

In [31]:
mel['Url_With_Coordinates'] = Url_With_Coordinates

In [41]:
mel['Url_With_Coordinates'].iloc[0].split('?center=')[1].split('&zoom=')[0].split('%2C')[0]

'-37.8386737'

In [42]:
[url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in mel['Url_With_Coordinates']]

IndexError: list index out of range

In [32]:
mel['lat'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in mel['Url_With_Coordinates']]

IndexError: list index out of range

In [43]:
mel[~mel.Url_With_Coordinates.str.contains('&zoom=')]

,ABN,Charity_Legal_Name,Address_Line_1,Address_Line_2,Address_Line_3,Town_City,State,Postcode,Country,Date_Organisation_Established,Charity_Size,Full_Address,Url,Url_With_Coordinates
38696,57533056318,Alternative Technology Association Inc.,L 1 Se 1 39 Little Collins St,,,Melbourne,Victoria,3000,Australia,,Large,L 1 Se 1 39 Little Collins St Melbourne,https://www.google.com/maps/search/L 1 Se 1 39...,//www.gstatic.com/images/branding/product/1x/m...
44052,64077478696,Women's Property Initiatives Ltd,L 1 22 William St,,,Melbourne,Victoria,3000,Australia,01/01/1997,Large,L 1 22 William St Melbourne,https://www.google.com/maps/search/L 1 22 Will...,//www.gstatic.com/images/branding/product/1x/m...
44158,64162498651,AUSTRALIAN INSTITUTE OF ARCHITECTS FOUNDATION,L 1 Se 1 41 Exhibition St,,,Melbourne,Victoria,3000,Australia,21/02/2013,Small,L 1 Se 1 41 Exhibition St Melbourne,https://www.google.com/maps/search/L 1 Se 1 41...,//www.gstatic.com/images/branding/product/1x/m...
54032,76140094882,Embark Australia Ltd,L 1 Se 1 39 Little Collins St,,,Melbourne,Victoria,3000,Australia,01/01/2009,Small,L 1 Se 1 39 Little Collins St Melbourne,https://www.google.com/maps/search/L 1 Se 1 39...,//www.gstatic.com/images/branding/product/1x/m...


In [44]:
mel = mel[mel.Url_With_Coordinates.str.contains('&zoom=')].copy()

In [49]:
mel['lat'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in mel['Url_With_Coordinates']]
mel['long'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1] for url in mel['Url_With_Coordinates']]

In [50]:
mel.head()

,ABN,Charity_Legal_Name,Address_Line_1,Address_Line_2,Address_Line_3,Town_City,State,Postcode,Country,Date_Organisation_Established,Charity_Size,Full_Address,Url,Url_With_Coordinates,lat,long
15,11004285956,Australian Nurses Memorial Centre,G Se 11 431 St Kilda Rd,,,Melbourne,Victoria,3004,Australia,31/12/1949,Small,G Se 11 431 St Kilda Rd Melbourne,https://www.google.com/maps/search/G Se 11 431...,https://maps.google.com/maps/api/staticmap?cen...,-37.8386737,144.97706
123,11128866419,Australian Community Philanthropy Ltd,L 15 1 Collins St,,,Melbourne,Victoria,3000,Australia,01/01/2007,Small,L 15 1 Collins St Melbourne,https://www.google.com/maps/search/L 15 1 Coll...,https://maps.google.com/maps/api/staticmap?cen...,-37.8139436,144.9734554
125,11129013305,Nafsika Stamoulis Hellenic Museum Limited,280 William St,,,Melbourne,Victoria,3000,Australia,01/01/2007,Large,280 William St Melbourne,https://www.google.com/maps/search/280 William...,https://maps.google.com/maps/api/staticmap?cen...,-37.8125657,144.9570751
126,11129790467,Doxa Community Club Incorporated,"Ground Floor, Suite 2, 355 Spencer Street",,,West Melbourne,VIC,3003,Australia,01/01/1992,Large,"Ground Floor, Suite 2, 355 Spencer Street We...",https://www.google.com/maps/search/Ground Floo...,https://maps.google.com/maps/api/staticmap?cen...,-37.8119391,144.950558
135,11137037677,Financial Basics Community Foundation Limited,L 7 525 Flinders St,,,Melbourne,Victoria,3000,Australia,01/01/2009,Small,L 7 525 Flinders St Melbourne,https://www.google.com/maps/search/L 7 525 Fli...,https://maps.google.com/maps/api/staticmap?cen...,-37.820758,144.9571827


In [ ]:
# https://towardsdatascience.com/using-python-and-selenium-to-get-coordinates-from-street-addresses-62706b6ac250